
# NVIDIA NIMs with Tool Calling for Agents

This notebook will use a [NVIDIA Llama 3.1 NIM](https://developer.nvidia.com/blog/supercharging-llama-3-1-across-nvidia-platforms/) with tool-calling agent capabilities in generative AI solutions. As mentioned in this [Introductory Blog on LLM Agents](https://developer.nvidia.com/blog/introduction-to-llm-agents/), agents can be described as AI systems that use LLMs to reason through a problem, create a plan to solve the problem, execute the plan with the help of a set of tools, and use memory to store meaningful context of the system state. 

The notebook is designed to provide an intro to merely one of the capabilities of agent systems: **tool calling**. 

**Tools** are interfaces that accept input, execute an action, and then return a result of that action in a structured output according to a pre-defined schema. They often encompass external API calls that the agent can use to perform tasks that go beyond the capabilities of the LLM, but do not have to be external API calls. For example, to get the current weather in San Diego, a weather tool might be used. Or to get the current score of the 49ers game, a generic web search tool or ESPN tool might be used. 

## What is NVIDIA NIM and How do They Support Tool Calling for Agents?
### What is NIM?
NIM supports models across domains like chat, embedding, and re-ranking models 
from the community as well as NVIDIA. These models are optimized by NVIDIA to deliver the best performance on NVIDIA 
accelerated infrastructure and deployed as a NIM, an easy-to-use, prebuilt containers that deploy anywhere using a single 
command on NVIDIA accelerated infrastructure. If you're new to NIMs with LangChain, check out the [documentation](https://python.langchain.com/v0.2/docs/integrations/providers/nvidia/).

Now, NIMs support tool calling, also known as "function calling" for models that have the aforementioned capability. 

This notebook will demonstrate a model that supports function calling, [Llama 3.1 8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct). 

### What does it mean for NIM to support tool usage?
In order to support tool usage in an agent workflow, first an LLM must be trained to detect when a function should be called and output a structured response like JSON that contains the function to be called and its arguments. 

Next, the model is packaged as a NIM, meaning it's optimized to deliver best performance on NVIDIA accelerated infrastructure and easy to deploy as well as use. This microservice packaging also uses OpenAI compatible APIs, so developers can build world-class generative AI agents with ease.

Let's see how to use tools in a couple of examples.

##  🔨 Tool Usage -- Web Search

Since a LLM does not have access to the most up-to-date information on the Internet, [Tavily Search](https://docs.tavily.com/docs/tavily-api/introduction) acts as a tool to provide a generative AI application with real-time online information.  Tavily is a search engmine that is optimized for AI developers and AI agents. A singular API call abstracts searching, scraping, filtering, and extracting relevant information from online sources. 

We'll enhance our NIM, [Llama 3.1-8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct), with Tavily search. 

Install pre-requesites. 

In [ ]:
%pip install -U langchain langgraph langchain-nvidia-ai-endpoints langchain-community langchain-openai tavily-python geocoder

If you're using NVIDIA hosted NIMs, you'll need to use an API key which you can setup below. Follow [NVIDIA NIMs LangChain documentation](https://python.langchain.com/v0.2/docs/integrations/chat/nvidia_ai_endpoints/) for more information on accessing and using NIMs. 

In [15]:
import getpass
import os

os.environ["NVIDIA_API_KEY"] = "nvapi-xxx"

Declare your model that supports tool calling. In this example, we use [Llama 3.1-8b-instruct](https://build.nvidia.com/meta/llama-3_1-8b-instruct). 

In [20]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

Initialize [Tavily Tool](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/)

Note that this requires an API key - they have a free tier, but if you don't have one or don't want to create one, you can always ignore this step or use a different tool. 

Once you create your API key, you will need to set it in the environment.

In [21]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = "tvly-xxx"

In [22]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Declare a single tool, Tavily search
tools = [TavilySearchResults(max_results=1)]

Create [ReAct agent](https://python.langchain.com/v0.2/docs/concepts/#react-agents), prebuilt in [LangGraph](https://langchain-ai.github.io/langgraph/#overview). 

In [23]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.callbacks.tracers import ConsoleCallbackHandler

prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)

/home/nvidia/test/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


Run agent; a callback is passed to provide more verbose output.

In [30]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'langchain definition'}`


[{'url': 'https://python.langchain.com/v0.2/docs/introduction/', 'content': "Introduction. LangChain is a framework for developing applications powered by large language models (LLMs).. LangChain simplifies every stage of the LLM application lifecycle: Development: Build your applications using LangChain's open-source building blocks, components, and third-party integrations.Use LangGraph to build stateful agents with first-class streaming and human-in-the-loop support."}]
Invoking: `tavily_search_results_json` with `{'query': 'langchain'}`


[{'url': 'https://python.langchain.com/v0.2/docs/introduction/', 'content': "LangChain is a framework for developing applications powered by large language models (LLMs). Learn how to use LangChain's open-source libraries, components, and integrations to build, deploy, and evaluate LLM applications and agents."}]Based on the i

{'input': 'What is langchain?',
 'output': 'Based on the information provided, LangChain is a framework for developing applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, including development, deployment, and evaluation.'}

## 🔨 Tool Usage -- Adding on a Custom Tool

Let's see how to [define a custom tool](https://python.langchain.com/v0.2/docs/how_to/custom_tools/) for your NIM agent and how it handles multiple tools.  

We'll enhance the NIM with Tavily search with some custom tools to determine a user's current location (based on IP address) and return a latitude and longitude. We will use these tools to have Tavily look up the weather in the user's current location.

First, let's create a custom tool to determine a user's location based off IP address. 

In [25]:
import geocoder
from langchain.tools import tool
from typing import Tuple

@tool
def get_current_location() -> list:
    """Return the current location of the user based on IP address"""
    loc = geocoder.ip('me')
    return loc.latlng    

Let's update the tools to use the Tavily tool delcared earlier and also add the `get_current_location` tool.

In [26]:
# Declare two tools: Tavily and custom get_current_location tool.
tools = [TavilySearchResults(max_results=1), get_current_location]

We already declared our LLM, so we don't need to redeclare it. However, we do want to update the agent to have the updated tools.

In [28]:
from langchain.globals import set_verbose
from langchain.callbacks.tracers import ConsoleCallbackHandler

set_verbose(True) # verbose output to follow function calling

query = "Search for the current weather information of my location?"
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `get_current_location` with `{}`


[35.7721, -78.6386]
Invoking: `get_current_location` with `{}`


[35.7721, -78.6386]
Invoking: `tavily_search_results_json` with `{'query': 'weather in the location 35.7721, -78.6386'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Raleigh', 'region': 'North Carolina', 'country': 'United States of America', 'lat': 35.77, 'lon': -78.64, 'tz_id': 'America/New_York', 'localtime_epoch': 1723484374, 'localtime': '2024-08-12 13:39'}, 'current': {'last_updated_epoch': 1723483800, 'last_updated': '2024-08-12 13:30', 'temp_c': 28.8, 'temp_f': 83.8, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 67, 'wind_dir': 'ENE', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.04, 'precip_in': 0.0, 'humidity': 48, 'cloud': 73, 'feelslike_c': 29.7, 'fe

{'input': 'Search for the current weather information of my location?',
 'output': 'The current weather information of your location is as follows: the location is Raleigh, North Carolina, United States of America, with a temperature of 28.8 degrees Celsius and a humidity of 48%. The weather condition is patchy rain nearby, with a wind speed of 6.5 km/h and a wind direction of ENE. The atmospheric pressure is 1017.0 mb, and the precipitation is 0.04 mm. The feels-like temperature is 29.7 degrees Celsius, and the heat index is also 29.7 degrees Celsius. The dew point is 16.9 degrees Celsius, and the visibility is 10.0 km. The UV index is 6.0, and the gust speed is 8.2 km/h.'}

In order to execute this query, first a tool to get the current location needs to be called. Then a tool to get the current weather at that location needs to be called. 
Finally, the result is returned to the user.

## Conclusion
You've now seen how to use NIMs to do tool calling, an important capability of agents. As mentioned earlier, tools are just one part of agent capabilities, so check out other notebook so see how tools can be used with othe techniques to create agent workflows.

If you're ready to explore more complicated agent workflows, check out [this blog](https://developer.nvidia.com/blog/build-an-agentic-rag-pipeline-with-llama-3-1-and-nvidia-nemo-retriever-nims/) on how to improve your RAG pipeline with agents with Llama 3.1 and NVIDIA NemMo Retriever NIMs.